In [3]:
# === Librerías necesarias ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === Parámetros base ===
tasa_descuento = 0.07
vida_util = 25
degradacion = 0.005
opex_frac = 0.015
capex_pv = 800     # USD/kW
capex_csp = 4000   # USD/kW

# === Función general para calcular LCOE (con degradación) ===
def calcular_lcoe(capex, opex_anual, produccion_anual, vida_util, tasa_descuento, degradacion):
    numerador = capex + sum([opex_anual / ((1 + tasa_descuento) ** t) for t in range(1, vida_util + 1)])
    denominador = sum([produccion_anual * ((1 - degradacion) ** (t-1)) / ((1 + tasa_descuento) ** t) for t in range(1, vida_util + 1)])
    return numerador / denominador

# === 1. Cálculo de LCOE para PV ===
df_pv = pd.read_csv('resumen_simulaciones_pvwatts_metrico.csv')
lcoe_pv = []

for _, row in df_pv.iterrows():
    capacidad = row['Capacidad (kW)']
    produccion = row['Producción Anual (kWh)']
    capex_total = capex_pv * capacidad
    opex_anual = opex_frac * capex_total
    lcoe = calcular_lcoe(capex_total, opex_anual, produccion, vida_util, tasa_descuento, degradacion)
    lcoe_pv.append(lcoe)

# Agregar y guardar resultados PV
df_pv['LCOE'] = lcoe_pv
df_pv['Tecnología'] = 'PV'
df_pv.to_csv('lcoe_pv.csv', index=False)
print('✅ LCOE PV guardado en lcoe_pv.csv')

# === 2. Cálculo de LCOE para CSP ===
df_csp = pd.read_csv('resultados_simulacion_csp_con_almacenamiento.csv')
lcoe_csp = []

for _, row in df_csp.iterrows():
    capacidad_mw = 35
    produccion = row['Producción Anual (MWh)'] * 1000
    capex_total = capex_csp * capacidad_mw * 1000
    opex_anual = opex_frac * capex_total
    lcoe = calcular_lcoe(capex_total, opex_anual, produccion, vida_util, tasa_descuento, degradacion)
    lcoe_csp.append(lcoe)

# Agregar y guardar resultados CSP
df_csp['LCOE'] = lcoe_csp
df_csp['Tecnología'] = 'CSP'
df_csp.to_csv('lcoe_csp.csv', index=False)
print('✅ LCOE CSP guardado en lcoe_csp.csv')

# === 3. Visualización comparativa PV vs CSP ===
df_all = pd.concat([
    df_pv[['País', 'LCOE', 'Tecnología']],
    df_csp[['País', 'LCOE', 'Tecnología']]
])

plt.figure(figsize=(10,6))
sns.boxplot(data=df_all, x='País', y='LCOE', hue='Tecnología')
plt.title('Comparación de LCOE por País y Tecnología')
plt.ylabel('LCOE (USD/kWh)')
plt.tight_layout()
plt.savefig('lcoe_comparativo_pv_csp.png')
plt.close()

# === 4. Análisis solo para PV ===
def crear_graficos_pv(resultados_pv):
    # Boxplot por país
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='País', y='LCOE', data=resultados_pv)
    plt.title('Distribución de LCOE por País')
    plt.ylabel('LCOE ($/kWh)')
    plt.savefig('lcoe_por_pais_boxplot.png')
    plt.close()

    # LCOE vs Capacidad
    plt.figure(figsize=(12, 6))
    for pais in resultados_pv['País'].unique():
        datos = resultados_pv[resultados_pv['País'] == pais]
        plt.plot(datos['Capacidad (kW)'], datos['LCOE'], 'o-', label=pais)
    plt.xlabel('Capacidad (kW)')
    plt.ylabel('LCOE ($/kWh)')
    plt.title('LCOE vs Capacidad por País')
    plt.legend()
    plt.grid(True)
    plt.savefig('lcoe_vs_capacidad.png')
    plt.close()

    # Heatmaps por país
    for pais in resultados_pv['País'].unique():
        datos = resultados_pv[resultados_pv['País'] == pais]
        pivot = datos.pivot_table(values='LCOE', index='Tilt (°)', columns='DC/AC Ratio', aggfunc='mean')
        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot, annot=True, fmt='.4f', cmap='YlOrRd')
        plt.title(f'LCOE por Tilt y DC/AC Ratio - {pais}')
        plt.savefig(f'lcoe_heatmap_{pais}.png')
        plt.close()

    # Barras de mejor configuración
    mejores = resultados_pv.loc[resultados_pv.groupby('País')['LCOE'].idxmin()]
    plt.figure(figsize=(12, 6))
    bars = plt.bar(mejores['País'], mejores['LCOE'], color='lightcoral')
    plt.title('LCOE Mínimo por País')
    plt.ylabel('LCOE ($/kWh)')
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.4f}', ha='center', va='bottom')
    plt.savefig('lcoe_minimo_por_pais.png')
    plt.close()

# Ejecutar generación de gráficos
crear_graficos_pv(df_pv)

# Mostrar listado final de imágenes generadas
print("\n📊 Gráficos generados y guardados:")
print("- lcoe_comparativo_pv_csp.png")
print("- lcoe_por_pais_boxplot.png")
print("- lcoe_vs_capacidad.png")
print("- lcoe_minimo_por_pais.png")
for pais in df_pv['País'].unique():
    print(f"- lcoe_heatmap_{pais}.png")


✅ LCOE PV guardado en lcoe_pv.csv
✅ LCOE CSP guardado en lcoe_csp.csv

📊 Gráficos generados y guardados:
- lcoe_comparativo_pv_csp.png
- lcoe_por_pais_boxplot.png
- lcoe_vs_capacidad.png
- lcoe_minimo_por_pais.png
- lcoe_heatmap_Chile.png
- lcoe_heatmap_China.png
- lcoe_heatmap_Rusia.png
